In [1]:
import requests
import pandas as pd

# 🔐 Replace this with your actual API key
API_KEY = "89c61046d8ed0ccf9ebfa3996ef443cc"

# 📡 API endpoint and parameters
url = "https://api.the-odds-api.com/v4/sports/boxing_boxing/odds"
params = {
    "apiKey": API_KEY,
    "regions": "us,uk,eu",       # Betting regions
    "markets": "h2h",            # "head-to-head" odds
    "oddsFormat": "decimal"      # You can also use 'american' or 'fractional'
}

# 🚀 Send the GET request
response = requests.get(url, params=params)

# 📦 Convert response to JSON
if response.status_code == 200:
    data = response.json()
    print(f"Received {len(data)} events.")
else:
    print("Error:", response.status_code, response.text)


Received 47 events.


In [4]:
# Print the first event to understand the structure
import json
print(json.dumps(data[0], indent=2))

{
  "id": "33ebcf4b4c433b51f81ed5375f9fe2e7",
  "sport_key": "boxing_boxing",
  "sport_title": "Boxing",
  "commence_time": "2025-08-16T15:00:00Z",
  "home_team": "Raymond Ford",
  "away_team": "Abraham Nova",
  "bookmakers": [
    {
      "key": "onexbet",
      "title": "1xBet",
      "last_update": "2025-08-08T18:24:20Z",
      "markets": [
        {
          "key": "h2h",
          "last_update": "2025-08-08T18:24:20Z",
          "outcomes": [
            {
              "name": "Abraham Nova",
              "price": 6.85
            },
            {
              "name": "Raymond Ford",
              "price": 1.11
            },
            {
              "name": "Draw",
              "price": 17.5
            }
          ]
        }
      ]
    },
    {
      "key": "virginbet",
      "title": "Virgin Bet",
      "last_update": "2025-08-08T18:24:20Z",
      "markets": [
        {
          "key": "h2h",
          "last_update": "2025-08-08T18:24:20Z",
          "outcomes": [
  

In [5]:
import pandas as pd

flat_data = []

for event in data:
    for bookmaker in event['bookmakers']:
        for market in bookmaker['markets']:
            for outcome in market['outcomes']:
                flat_data.append({
                    'event_id': event['id'],
                    'commence_time': event['commence_time'],
                    'fighter': outcome['name'],
                    'market': market['key'],
                    'decimal_odds': outcome['price'],
                    'bookmaker': bookmaker['title'],
                    'last_update': bookmaker['last_update'],
                    'home_team': event.get('home_team'),
                    'away_team': event.get('away_team')
                })

odds_df = pd.DataFrame(flat_data)


In [6]:
import sqlite3

# Connect to SQLite
conn = sqlite3.connect("boxing_odds_staging.db")
cursor = conn.cursor()

# Explicitly truncate (delete all rows)
cursor.execute("DELETE FROM raw_odds;")
conn.commit()

# Insert fresh data
odds_df.to_sql("raw_odds", conn, if_exists="append", index=False)

# Close connection
conn.close()


In [7]:
conn = sqlite3.connect("boxing_odds_staging.db")
pd.read_sql("SELECT * FROM raw_odds LIMIT 100", conn)

,event_id,commence_time,fighter,market,decimal_odds,bookmaker,last_update,home_team,away_team
0,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Abraham Nova,h2h,6.85,1xBet,2025-08-08T18:24:20Z,Raymond Ford,Abraham Nova
1,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Raymond Ford,h2h,1.11,1xBet,2025-08-08T18:24:20Z,Raymond Ford,Abraham Nova
2,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Draw,h2h,17.50,1xBet,2025-08-08T18:24:20Z,Raymond Ford,Abraham Nova
3,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Abraham Nova,h2h,6.25,Virgin Bet,2025-08-08T18:24:20Z,Raymond Ford,Abraham Nova
4,33ebcf4b4c433b51f81ed5375f9fe2e7,2025-08-16T15:00:00Z,Raymond Ford,h2h,1.15,Virgin Bet,2025-08-08T18:24:20Z,Raymond Ford,Abraham Nova
...,...,...,...,...,...,...,...,...,...
95,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,Draw,h2h,14.00,LeoVegas,2025-08-08T18:24:20Z,Filip Hrgović,David Adeleye
96,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,David Adeleye,h2h,5.25,BetUS,2025-08-08T18:24:21Z,Filip Hrgović,David Adeleye
97,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,Filip Hrgović,h2h,1.16,BetUS,2025-08-08T18:24:21Z,Filip Hrgović,David Adeleye
98,f1cdd1ab69a4e0a0a075eefae1295763,2025-08-16T15:00:00Z,David Adeleye,h2h,5.35,Betsson,2025-08-08T18:24:21Z,Filip Hrgović,David Adeleye
